In [ ]:
# 🔧 Setup: Installation und Importe
# Führen Sie diese Zelle aus, um alle notwendigen Libraries zu installieren

import warnings
warnings.filterwarnings('ignore')

# Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

# Machine Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Computer Vision
import cv2
from scipy import signal
import skimage
from skimage import filters, feature

# Streamlit (für interaktive Apps)
import streamlit as st
from ipywidgets import interact, widgets

# Konfiguration
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Alle Libraries erfolgreich importiert!")
print(f"📊 TensorFlow Version: {tf.__version__}")
print(f"🖼️ OpenCV Version: {cv2.__version__}")

# GPU Check
if tf.config.list_physical_devices('GPU'):
    print("🚀 GPU verfügbar!")
else:
    print("💻 CPU wird verwendet")

# [Nur Colab] Diese Zellen müssen nur auf *Google Colab* ausgeführt werden und installieren Packete und Daten
!wget -q https://raw.githubusercontent.com/KI-Campus/AMALEA/master/requirements.txt && pip install --quiet -r requirements.txt
!wget --quiet "https://github.com/KI-Campus/AMALEA/releases/download/data/data.zip" && unzip -q data.zip
!wget --quiet "https://github.com/KI-Campus/AMALEA/releases/download/images/images.zip" && unzip -q images.zip

# 👁️ 06.1 CNN Grundlagen - Convolutional Neural Networks verstehen

**Data Analytics & Big Data - Woche 6.1**  
*IU Internationale Hochschule*

---

## 🎯 Lernziele

Nach diesem Notebook können Sie:
- ✅ **CNN-Architektur** verstehen (Convolutional, Pooling, Dense Layers)
- ✅ **Bildfilter** anwenden und deren Wirkung verstehen  
- ✅ **Feature Maps** visualisieren und interpretieren
- ✅ **Streamlit-App** für Computer Vision entwickeln
- ✅ **TensorFlow/Keras** für CNN-Implementierung nutzen

---

## 📚 Was sind CNNs?

**Convolutional Neural Networks (CNNs)** sind spezialisierte neuronale Netze für die **Bildverarbeitung**. 

### 🔑 Kernkonzepte:
1. **Convolution (Faltung)** - Erkennt lokale Features wie Kanten
2. **Pooling** - Reduziert Bildgröße, behält wichtige Information  
3. **Feature Maps** - Zeigen welche Merkmale das Netz erkannt hat
4. **Hierarchical Learning** - Von einfachen Kanten zu komplexen Objekten

### 🏗️ CNN-Architektur:
```
Input Image → Conv2D → ReLU → MaxPool → Conv2D → ReLU → MaxPool → Flatten → Dense → Output
```

![CNN feedforward](images/stanford_cnn.jpg "CNN_feedforward")

*Quelle: Stanford CS231n*

## 🧠 Warum CNNs für Computer Vision?

### 🤔 Das Problem mit normalen Neural Networks

**Normale Fully-Connected Networks** für Bilder haben massive Probleme:

```python
# Beispiel: 28x28 Pixel Bild (MNIST)
input_size = 28 * 28 * 1  # = 784 Parameter pro Bild
hidden_layer = 128        # = 784 * 128 = 100,352 Gewichte nur für erste Schicht!

# Bei 224x224 RGB Bildern (ImageNet):
input_size = 224 * 224 * 3  # = 150,528 Parameter
# → Millionen von Gewichten schon in der ersten Schicht!
```

### 💡 Die CNN-Lösung

**CNNs nutzen 3 Schlüsselkonzepte:**

#### 1. 🔍 **Local Connectivity (Lokale Verbindungen)**
- Jedes Neuron schaut nur auf einen kleinen Bildbereich (z.B. 3x3 Pixel)
- Reduziert Parameter drastisch

#### 2. 🔄 **Weight Sharing (Gewichtsteilen)**  
- Derselbe Filter wird über das gesamte Bild angewendet
- Ein 3x3 Filter hat nur 9 Parameter (statt Millionen!)

#### 3. 📐 **Translation Invariance (Positionsunabhängigkeit)**
- Erkennt Objekte unabhängig von ihrer Position im Bild
- Katze oben links = Katze unten rechts

### 🏗️ CNN-Architektur im Detail

```
Input (28x28x1) 
    ↓ Conv2D(32 filters, 3x3) + ReLU
Feature Maps (26x26x32)
    ↓ MaxPooling(2x2) 
Reduced Maps (13x13x32)
    ↓ Conv2D(64 filters, 3x3) + ReLU
Feature Maps (11x11x64)
    ↓ MaxPooling(2x2)
Final Maps (5x5x64)
    ↓ Flatten
Vector (1600)
    ↓ Dense(128) + ReLU
    ↓ Dense(10) + Softmax
Output (10 classes)
```

## 🔍 Bildfilter verstehen - Von klassisch zu CNN

### 🎯 Lernziel
Bevor wir CNNs trainieren, verstehen wir **wie Filter funktionieren** durch klassische Computer Vision Filter.

### 📊 Filter-Konzept
Ein **Filter (Kernel)** ist eine kleine Matrix (z.B. 3x3), die über ein Bild "gefaltet" wird:

```python
# Beispiel 3x3 Filter
filter_3x3 = [
    [a, b, c],
    [d, e, f], 
    [g, h, i]
]

# Convolution Operation:
output_pixel = (pixel_area * filter).sum()
```

### 🛠️ Filter-Typen die wir testen werden:

1. **🌫️ Blur Filter (Mean)** - Bild unscharf machen
2. **⚡ Edge Detection (Prewitt/Sobel)** - Kanten finden  
3. **🔄 Sharpening (Laplace)** - Bild schärfen
4. **🎨 Custom Filters** - Eigene Effekte

**💡 Der Clou:** CNNs lernen diese Filter automatisch!

### Importe

In [ ]:
# 📚 Spezielle Importe für Bildverarbeitung
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import figure
import numpy as np
from scipy import misc, signal
from PIL import Image
import cv2

# Matplotlib Konfiguration für bessere Plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Bildverarbeitungs-Libraries importiert!")

# Test: Verfügbare Beispielbilder
try:
    # Scipy Beispielbild
    ascent = misc.ascent()
    print(f"📸 Beispielbild geladen: {ascent.shape}")
    
    # OpenCV Test
    print(f"🔧 OpenCV Version: {cv2.__version__}")
    
except Exception as e:
    print(f"⚠️ Fehler beim Laden: {e}")
    print("💡 Tipp: Installieren Sie scipy und opencv-python")

In [ ]:
#Load Ascent image from scipy
ascent = misc.ascent()

# Moderne Visualisierung
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Original Bild
axes[0].imshow(ascent, cmap='gray', interpolation='nearest')
axes[0].set_title('🏔️ Original Bild (Ascent)', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Histogram
axes[1].hist(ascent.flatten(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[1].set_title('📊 Pixel-Verteilung', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Pixel-Intensität (0-255)')
axes[1].set_ylabel('Anzahl Pixel')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Bildinformationen
print(f"📏 Bildgröße: {ascent.shape}")
print(f"📊 Datentyp: {ascent.dtype}")
print(f"📈 Min/Max Werte: {ascent.min()} / {ascent.max()}")
print(f"🎯 Durchschnitt: {ascent.mean():.1f}")

# Fun Fact
print(f"\n💡 Fun Fact: Dieses {ascent.shape[0]}x{ascent.shape[1]} Bild hat {ascent.size:,} Pixel!")
print(f"💾 Speicherbedarf: {ascent.nbytes:,} Bytes ({ascent.nbytes/1024:.1f} KB)")

#### Mean-Filter

Der erste Filter, den wir anwenden werden, heißt "Mean-Filter". Der Mean-Filter ersetzt einen Pixelwert durch den Mittelwert der Werte, die sich in der Nachbarschaft des Pixels $9\times 9$ befinden. Das heißt, wir verwenden alle benachbarten Pixel sowie die Werte des zu ersetzenden Pixels, dann berechnen wir den Mittelwert dieser neun Werte und verwenden das Ergebnis als neuen Pixelwert.

## 🎮 Interaktive Filter-Experimente

### 💻 Jupyter Widgets für Live-Experimente

Wir können mit **ipywidgets** interaktiv verschiedene Filter testen:

In [ ]:
# 🎮 Interaktive Filter-Demo

def apply_filter_interactive(filter_type='Original', blur_size=3):
    """
    Interaktive Funktion zum Testen verschiedener Filter
    """
    # Filter auswählen
    if filter_type == 'Original':
        result = ascent
    elif filter_type == 'Blur':
        kernel_size = max(3, blur_size)
        if kernel_size % 2 == 0:  # Kernel muss ungerade sein
            kernel_size += 1
        blur_kernel = np.ones((kernel_size, kernel_size)) / (kernel_size**2)
        result = signal.convolve2d(ascent, blur_kernel, boundary='symm', mode='same')
    elif filter_type == 'Edge X':
        result = np.abs(signal.convolve2d(ascent, sobel_x, boundary='symm', mode='same'))
    elif filter_type == 'Edge Y':
        result = np.abs(signal.convolve2d(ascent, sobel_y, boundary='symm', mode='same'))
    elif filter_type == 'Sharpen':
        result = signal.convolve2d(ascent, sharpen, boundary='symm', mode='same')
        result = np.clip(result, 0, 255)  # Werte begrenzen
    elif filter_type == 'Laplace':
        result = signal.convolve2d(ascent, laplace, boundary='symm', mode='same')
        result = np.abs(result)  # Absolutwerte für bessere Sichtbarkeit
    
    # Visualisierung
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(ascent, cmap='gray')
    plt.title('🏔️ Original', fontsize=14, fontweight='bold')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(result, cmap='gray')
    plt.title(f'{filter_type} Filter', fontsize=14, fontweight='bold')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiken
    print(f"📊 {filter_type} Filter Statistiken:")
    print(f"   Min: {result.min():.1f}, Max: {result.max():.1f}")
    print(f"   Mean: {result.mean():.1f}, Std: {result.std():.1f}")

# Widget erstellen
filter_widget = interact(
    apply_filter_interactive,
    filter_type=widgets.Dropdown(
        options=['Original', 'Blur', 'Edge X', 'Edge Y', 'Sharpen', 'Laplace'],
        value='Original',
        description='🔧 Filter:'
    ),
    blur_size=widgets.IntSlider(
        value=3, min=3, max=15, step=2,
        description='Blur Size:'
    )
)

print("🎮 Interaktive Filter-Demo geladen!")
print("💡 Tipp: Probieren Sie verschiedene Filter und Blur-Größen aus!")

In [ ]:
# Some 3x3 filter matrices used in computer vision

# Mean-filter
mean_3x3 = (1/9) * np.ones([3, 3])
mean_9x9 = (1/81) * np.ones([9, 9])  # Stärkerer Blur-Effekt

# Approximation of the gradient
# Prewitt Filter - findet Kanten in x/y Richtung
prewitt_x = np.array([[-1, 0, 1], 
                      [-1, 0, 1], 
                      [-1, 0, 1]])

prewitt_y = np.array([[-1, -1, -1], 
                      [ 0,  0,  0], 
                      [ 1,  1,  1]])

# Sobel Filter - bessere Kantendetection (gewichtet)
sobel_x = np.array([[-1, 0, 1], 
                    [-2, 0, 2], 
                    [-1, 0, 1]])

sobel_y = np.array([[-1, -2, -1], 
                    [ 0,  0,  0], 
                    [ 1,  2,  1]])

# Second-order derivation 
# Laplace Filter - hebt Details hervor
laplace = np.array([[ 0, -1,  0], 
                    [-1,  4, -1], 
                    [ 0, -1,  0]])

# Enhanced Sharpening
sharpen = np.array([[ 0, -1,  0], 
                    [-1,  5, -1], 
                    [ 0, -1,  0]])

# 📊 Filter visualisieren
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
filters = [
    (mean_3x3, "🌫️ Blur (3x3)"),
    (prewitt_x, "⚡ Prewitt X"),
    (prewitt_y, "⚡ Prewitt Y"), 
    (sobel_x, "⚡ Sobel X"),
    (sobel_y, "⚡ Sobel Y"),
    (laplace, "🔍 Laplace"),
    (sharpen, "✨ Sharpen"),
    (mean_9x9[:3,:3], "🌫️ Blur (9x9)")  # Zeige nur 3x3 Ausschnitt
]

for i, (filter_matrix, title) in enumerate(filters):
    row, col = i // 4, i % 4
    im = axes[row, col].imshow(filter_matrix, cmap='RdBu', vmin=-2, vmax=2)
    axes[row, col].set_title(title, fontsize=12, fontweight='bold')
    
    # Werte in die Zellen schreiben
    for (j, k), val in np.ndenumerate(filter_matrix):
        axes[row, col].text(k, j, f'{val:.1f}', ha='center', va='center', 
                           color='white' if abs(val) > 1 else 'black', fontweight='bold')
    
    axes[row, col].set_xticks([])
    axes[row, col].set_yticks([])

plt.tight_layout()
plt.show()

print("✅ Filter erfolgreich definiert!")
print("\n💡 Wie Filter funktionieren:")
print("1. Filter-Matrix wird über Bild 'gefaltet' (convolved)")  
print("2. Jeder Pixel wird durch gewichtete Summe seiner Nachbarn ersetzt")
print("3. Verschiedene Filter erkennen verschiedene Features")

In [ ]:
# 🌫️ BLUR FILTER - Mean Filter anwenden

# Verschiedene Blur-Stärken testen
blur_light = signal.convolve2d(ascent, mean_3x3, boundary='symm', mode='same')
blur_heavy = signal.convolve2d(ascent, mean_9x9, boundary='symm', mode='same')

# Moderne Vergleichsvisualisierung
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

images = [
    (ascent, "🏔️ Original", "gray"),
    (blur_light, "🌫️ Blur (3x3)", "gray"),  
    (blur_heavy, "🌫️ Blur (9x9)", "gray")
]

for i, (img, title, cmap) in enumerate(images):
    axes[i].imshow(img, cmap=cmap, interpolation='nearest')
    axes[i].set_title(title, fontsize=14, fontweight='bold')
    axes[i].axis('off')
    
    # Info-Text hinzufügen
    axes[i].text(10, 30, f"Min: {img.min():.0f}\nMax: {img.max():.0f}\nMean: {img.mean():.1f}", 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
                fontsize=10)

plt.tight_layout()
plt.show()

# 📊 Detailvergleich - Bildbereich vergrößern
region = slice(200, 300), slice(200, 300)  # 100x100 Pixel Ausschnitt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, (img, title, cmap) in enumerate(images):
    axes[i].imshow(img[region], cmap=cmap, interpolation='nearest')
    axes[i].set_title(f"{title} - Detail", fontsize=12)
    axes[i].axis('off')

plt.suptitle("🔍 Detailvergleich: Blur-Effekt", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📝 Mean Filter Analyse:")
print("• 3x3 Filter: Leichter Blur-Effekt, Details bleiben erkennbar")
print("• 9x9 Filter: Starker Blur-Effekt, viele Details gehen verloren")
print("• Anwendung: Rauschunterdrückung, künstlerische Effekte")
print("\n💡 In CNNs: Das Netzwerk lernt automatisch, welche 'Blur-Stärke' optimal ist!")

Wie Sie sehen können, wird dadurch das Bild unscharf.

#### Prewitt-Filter

Das Ziel des Prewitt-Filters ist, den Pixel-Wert durch seine Ableitung, d.h. die Änderung im Farbwert, zu ersetzen. Auch hier kommt eine $9\times 9$-Nachbarschaft zum Einsatz. Da Bilder i.d.R. 2-dimensional sind (x- und y-Achse), kann die Ableitung sowohl der einen als auch der anderen Dimension berechnet werden. Praktisch bedeutet dies, dass es zwei Prewitt-Filter gibt, einen in x- und einen in y-Richtung. Außerdem korrespondieren größere Änderungen im Pixelwert (d.h. der Wert der Ableitung ist größer) mit Kanten im Bild, weshalb die Prewitt-Filter auch als Kantendetektoren bezeichnet werden.

In [5]:
ascent_x_prew = signal.convolve2d(ascent, prewitt_x, boundary='symm', mode='same')
ascent_x_prew = np.absolute(ascent_x_prew)
fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.set_title('Ergebnis nach Anwendung des Prewitt-Filters in x-Richtung', fontsize = 15)
ax.imshow(ascent_x_prew, interpolation='nearest', cmap='gray')
plt.tight_layout()

In [6]:
ascent_y_prew = signal.convolve2d(ascent, prewitt_y, boundary='symm', mode='same')
ascent_y_prew = np.absolute(ascent_y_prew)
fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.set_title('Ergebnis nach Anwendung des Prewitt-Filters in y-Richtung', fontsize = 15)
ax.imshow(ascent_y_prew, interpolation='nearest', cmap='gray')
plt.tight_layout()

Auf den beiden Bildern sieht man, dass die entsprechenden Kanten in entweder x- oder y-Richtung extrahiert wurden. In der praktischen Anwendung, möchte man aber meist alle Kanten extrahieren - unabhängig von der Richtung. In diesem Fall werden schlichtweg die beiden Bilder addiert und anschließend durch 2 geteilt. Das Teilen durch 2 ist notwendig, damit die Pixelwert im Bereich 0 bis 255 verbleiben. 

In [7]:
x_y_prew = (ascent_y_prew + ascent_x_prew) / 2

fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.set_title('Ergebnis nach Anwendung des Prewitt-Filters in x- & y-Richtung', fontsize = 15)
ax.imshow(x_y_prew, interpolation='nearest', cmap='gray')
plt.tight_layout()

<div class="alert alert-block alert-success">
<b>Frage 5.1.1:</b> Welcher lineare Prewitt-Filter wird benötigt, um horizontale Kanten hervorzuheben? 
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### Gesichtserkennung

Im folgenden wird die im Video erwähnte Gesichtserkennung skizziert.

Zu Beginn erstmal ein Bild mit Gesichtern:

In [8]:
import matplotlib.pyplot as plt
from PIL import Image

img = Image.open('images/Gesichter.jpg')

plt.figure(figsize=(15,10))
plt.imshow(img)
plt.show()


Nun kann mithilfe einer Gesichtserkennung, die ensprechenden Bounding Boxen extrahiert werden.

In [9]:
img = Image.open('images/Gesichtserkennung_1_Bounding_Boxes.png')

plt.figure(figsize=(15,10))
plt.imshow(img)
plt.show()

Und abschließend die Landmarks:

In [10]:
img = Image.open('images/Gesichtserkennung_2_Landmarks.png')

plt.figure(figsize=(15,10))
plt.imshow(img)
plt.show()

Nun können die Gesichter extrahiert werden

In [11]:
import os

path = './images/landmarks/face/'
for file in os.listdir(path):
    if ".png" in file:
        img = Image.open(os.path.join(path, file))
        plt.figure()
        plt.imshow(img)
        plt.show()

Nun können die Landmarken extrahiert werden:

In [12]:
import os

path = './images/landmarks/landmarks/'
for file in os.listdir(path):
    if ".png" in file:
        img = Image.open(os.path.join(path, file))
        plt.figure()
        plt.imshow(img)
        plt.show()

Und zuletzt werden die Gesichter transformiert, sodass alle gleich große (96x96 Pixel) sind und mittig liegen. Somit können die Landmarken (z.B. Augen) immer an der gleichen Stelle im Bild auftauchen. 

In [13]:
import os

path = './images/landmarks/transformed/'
for file in os.listdir(path):
    if ".png" in file:
        img = Image.open(os.path.join(path, file))
        plt.figure()
        plt.imshow(img)
        plt.show()


---

## 🚀 Von klassischen Filtern zu modernen CNNs

### 💡 Der Durchbruch: Lernbare Filter

Wir haben gesehen, wie **handgeschriebene Filter** funktionieren. CNNs machen dasselbe - nur **automatisch**!

### 🏗️ Modernes CNN mit TensorFlow/Keras

Lassen Sie uns ein einfaches CNN für MNIST-Ziffenerkennung erstellen:

In [ ]:
# 🤖 Modernes CNN mit TensorFlow/Keras

# MNIST Daten laden
print("📥 MNIST Dataset laden...")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Daten vorbereiten
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f"✅ Training: {x_train.shape}, Test: {x_test.shape}")

# CNN Modell erstellen
model = keras.Sequential([
    # 1. Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv1'),
    layers.MaxPooling2D((2, 2), name='pool1'),
    
    # 2. Convolutional Block  
    layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
    layers.MaxPooling2D((2, 2), name='pool2'),
    
    # 3. Dense Layers
    layers.Flatten(name='flatten'),
    layers.Dense(128, activation='relu', name='dense1'),
    layers.Dropout(0.2, name='dropout'),
    layers.Dense(10, activation='softmax', name='output')
])

# Modell kompilieren
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

# Modell-Architektur anzeigen
print("\n🏗️ CNN Architektur:")
model.summary()

# Visualisierung der Architektur
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, 
                          to_file='cnn_architecture.png', dpi=150)

# Plot anzeigen
try:
    from IPython.display import Image as IPImage
    display(IPImage('cnn_architecture.png'))
except:
    print("📊 Architektur in 'cnn_architecture.png' gespeichert")

print("\n💡 CNN vs. Fully-Connected Vergleich:")
print("🔹 CNN Parameter: {:,}".format(model.count_params()))

# Vergleich mit Fully-Connected
fc_params = (28*28) * 128 + 128 * 10  # Nur erste und letzte Schicht
print(f"🔹 Equivalent FC Parameter: {fc_params:,}")
print(f"🚀 Parameter-Reduktion: {(1 - model.count_params()/fc_params)*100:.1f}%")

In [ ]:
# 🎯 CNN Training (Quick Demo)

print("🏃‍♂️ Schnelles Training (1 Epoche für Demo)...")

# Kleine Stichprobe für schnelles Training
x_sample = x_train[:1000]
y_sample = y_train[:1000]

# Training
history = model.fit(
    x_sample, y_sample,
    epochs=1,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Test auf paar Beispielen
test_loss, test_acc = model.evaluate(x_test[:100], y_test[:100], verbose=0)
print(f"\n📊 Test Accuracy (100 Samples): {test_acc:.3f}")

print("✅ CNN erfolgreich trainiert!")

In [ ]:
# 🔍 Feature Maps Visualisierung - Das Herz von CNNs!

def visualize_feature_maps(model, input_image, layer_names=['conv1', 'conv2']):
    """
    Visualisiert Feature Maps von CNN Layern
    """
    # Model für Feature Extraction erstellen
    layer_outputs = [model.get_layer(name).output for name in layer_names]
    activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)
    
    # Feature Maps berechnen
    activations = activation_model.predict(input_image[np.newaxis, ...])
    
    for layer_idx, (layer_name, activation) in enumerate(zip(layer_names, activations)):
        # Anzahl Feature Maps
        n_features = activation.shape[-1]
        n_cols = 8
        n_rows = n_features // n_cols + (1 if n_features % n_cols else 0)
        
        # Plot erstellen
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 2*n_rows))
        fig.suptitle(f'🔍 Feature Maps - {layer_name.upper()} Layer', 
                     fontsize=16, fontweight='bold')
        
        for i in range(n_features):
            row, col = i // n_cols, i % n_cols
            if n_rows == 1:
                ax = axes[col] if n_cols > 1 else axes
            else:
                ax = axes[row, col]
                
            # Feature Map anzeigen
            feature_map = activation[0, :, :, i]
            im = ax.imshow(feature_map, cmap='viridis', interpolation='nearest')
            ax.set_title(f'Filter {i+1}', fontsize=10)
            ax.axis('off')
        
        # Leere Subplots verstecken
        for i in range(n_features, n_rows * n_cols):
            row, col = i // n_cols, i % n_cols
            if n_rows == 1:
                ax = axes[col] if n_cols > 1 else axes
            else:
                ax = axes[row, col]
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"📊 {layer_name} Layer: {activation.shape} -> {n_features} Feature Maps")

# Beispielbild auswählen
test_image = x_test[0]  # Erste Testziffer

# Original Bild anzeigen
plt.figure(figsize=(6, 6))
plt.imshow(test_image.squeeze(), cmap='gray')
plt.title('🔢 Original MNIST Ziffer', fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()

# Vorhersage
prediction = model.predict(test_image[np.newaxis, ...])
predicted_class = np.argmax(prediction)
confidence = np.max(prediction) * 100

print(f"🎯 Vorhersage: Ziffer {predicted_class} (Confidence: {confidence:.1f}%)")

# Feature Maps visualisieren
print("\n🔍 Feature Maps Analyse:")
print("Diese zeigen, was das CNN 'sieht' in jedem Layer...")
visualize_feature_maps(model, test_image)

---

## 🎮 Streamlit Integration

### 💻 Interaktive CNN Filter App

Sie können die **Streamlit App** parallel zu diesem Notebook ausführen:

```bash
# Im Terminal ausführen:
cd 06_Computer_Vision_NLP
streamlit run 06_01_streamlit_cnn_filter.py
```

### 🌟 Features der App:
- ✅ **Interaktive Filter-Tests** mit eigenen Bildern
- ✅ **Live Parameter-Anpassung** 
- ✅ **Filter-Kernel Visualisierung**
- ✅ **Bildstatistiken** in Echtzeit
- ✅ **Download-Funktion** für gefilterte Bilder

---

## 📚 Zusammenfassung & Key Takeaways

### ✅ Was Sie gelernt haben:

#### 1. 🔍 **Klassische Bildfilter**
- **Mean Filter:** Blur-Effekte durch Mittelwertbildung
- **Sobel/Prewitt:** Kanten-Detection durch Gradient-Berechnung  
- **Laplace:** Detail-Enhancement durch 2. Ableitung
- **Custom Filter:** Eigene 3x3 Kernel erstellen

#### 2. 🧠 **CNN Grundkonzepte**
- **Local Connectivity:** Nur lokale Pixel-Verbindungen
- **Weight Sharing:** Gleicher Filter über ganzes Bild
- **Translation Invariance:** Positionsunabhängige Erkennung
- **Hierarchical Learning:** Von einfach zu komplex

#### 3. 🚀 **Praktische CNN Implementation**
- **TensorFlow/Keras** für moderne CNNs
- **Feature Maps** visualisieren und verstehen
- **Parameter-Effizienz** vs. Fully-Connected Networks
- **MNIST Klassifikation** als Proof-of-Concept

### 💡 **Verbindung Filter ↔ CNN:**

```python
# Klassischer Filter (handgeschrieben)
sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
filtered = signal.convolve2d(image, sobel_x)

# CNN Filter (automatisch gelernt)
conv_layer = layers.Conv2D(32, (3, 3), activation='relu')
# Das Netzwerk lernt 32 verschiedene 3x3 Filter automatisch!
```

### 🎯 **Nächste Schritte:**

In den kommenden Notebooks vertiefen wir:
- **06.2:** Computer Vision Anwendungen (Objekterkennung, Segmentierung)
- **06.3:** Data Augmentation für bessere Modelle
- **06.4:** Transfer Learning mit vortrainierten Modellen

---

### 🏆 **Challenge für Sie:**

1. **📊 Experimentieren Sie** mit der Streamlit App - probieren Sie verschiedene Filter!
2. **🔧 Modifizieren Sie** das CNN-Modell (mehr Layer, andere Aktivierungen)
3. **📸 Testen Sie** eigene Bilder mit den klassischen Filtern
4. **🤔 Überlegen Sie:** Welche Filter würde ein CNN für Ihr Anwendungsgebiet lernen?

**💪 Advanced:** Implementieren Sie einen eigenen Filter in der Streamlit App!